<a href="https://colab.research.google.com/github/raheelam98/LangGraph/blob/main/chatbot_langgraph_quick_start%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)

[Quick Start](https://langchain-ai.github.io/langgraph/tutorials/introduction/)

**Chatbot**

00_quickstart_part1_to_part7.ipynb

[Chatbot - LangGraph Quick Start - rm Github](https://github.com/raheelam98/LangGraph/blob/main/langchain_ecosystem/langgraph/chatbot/docs/00_quickstart_part1_to_part7.ipynb)



In [ ]:
# install the required packages:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai

In [ ]:
# API Keys
# Get the GEMINI API key from user data
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

In [ ]:
# check each set is running
import os
# from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"

In [ ]:
# Initialize the ChatGoogleGenerativeAI with the Gemini model

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model to use
    max_retries=2,
    api_key=gemini_api_key    # Provide the Google API key for authentication
)

In [ ]:
llm.invoke("hello")

In [ ]:
# Invoke the LLM with a query
result = llm.invoke("Rain is expected in Karachi starting today?")
result

We'll use [LangSmith](https://docs.smith.langchain.com/) for [tracing](https://docs.smith.langchain.com/concepts/tracing).

In [ ]:
import os
from google.colab import userdata
LANGSMITH_API_KEY= userdata.get('LANGSMITH_API_KEY')

**Chatbot with Tools :** Integrate a web search tool into the bot.

First, install the requirements to use the [Tavily Search Engine](https://python.langchain.com/docs/integrations/tools/tavily_search/), and set your [TAVILY_API_KEY](https://tavily.com/).

In [ ]:
# Chatbot with Tools : Integrate a web search tool into the bot.
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [ ]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]
#tool.invoke("What's a 'node' in LangGraph?")
tool.invoke("Rain expected in Karachi this week")

## Part 3: Adding Memory to the Chatbot

**Provide a `checkpointer` when compiling the graph and a `thread_id` when calling your graph, LangGraph automatically saves the state after each step. When you invoke the graph again using the same `thread_id`, the graph loads its saved state, allowing the chatbot to pick up where it left off.**

**rm notes**

**Integrating memory checkpointing**

**`graph = graph_builder.compile(checkpointer=MemorySaver()) `** compiles the graph with memory checkpointing, allowing it to save and restore the state of the graph's nodes during its execution.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

Now you can interact with your bot! First, pick a thread to use as the key for this conversation.

In [ ]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

Next, call your chat bot.

In [ ]:
# Start conversation
user_input = "Hi there! My name is Bushra."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

Note: The config was provided as the second positional argument when calling our graph. It importantly is not nested within the graph inputs ({'messages': []}).

Let's ask a followup: see if it remembers your name.

In [ ]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

## Part 4: Human-in-the-loop

LangGraph supports `human-in-the-loop` workflows in a number of ways. In this section, we will use LangGraph's `interrupt_before` functionality to always break the tool node.

In [ ]:
#  compile the graph, specifying to interrupt_before the tools node.
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ tools, if desired.
    # interrupt_after=["tools"]
)

In [ ]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()